In [ ]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
df_RFM_histo_cleaned = dataiku.Dataset("df_RFM_histo_cleaned")
rfm_monthly = df_RFM_histo_cleaned.get_dataframe()

# Convertis la période en datetime pour extraire le semestre
rfm_monthly['periode'] = pd.to_datetime(rfm_monthly['periode'].astype(str) + '-01')
rfm_monthly['semester'] = rfm_monthly['periode'].dt.year.astype(str) + '-S' + ((rfm_monthly['periode'].dt.month - 1) // 6 + 1).astype(str)

# Définis un seuil pour la rétention (ex: 30 jours)
retention_threshold = 30

# Calcule le taux de rétention global par semestre
#retention_df = rfm_monthly.groupby('semester').agg(
#    total_customers=('customer_id', 'nunique'),
#    active_customers=('customer_id', lambda x: ((rfm_monthly.loc[x.index, 'recency'] < retention_threshold) &
#                                              (rfm_monthly.loc[x.index, 'period'] == rfm_monthly.loc[x.index, 'period'].max())).sum())
#).reset_index()
#retention_df['retention_rate'] = (retention_df['active_customers'] / retention_df['total_customers']) * 100

# Calcule le taux de rétention par segment et par semestre
segment_retention_df = rfm_monthly.groupby(['semester', 'segment']).agg(
    total_customers=('client_id', 'nunique'),
    active_customers=('client_id', lambda x: ((rfm_monthly.loc[x.index, 'recency'] < retention_threshold) &
                                                  (rfm_monthly.loc[x.index, 'periode'] == rfm_monthly.loc[x.index, 'periode'].max())).sum())
).reset_index()
segment_retention_df['retention_rate'] = (segment_retention_df['active_customers'] / segment_retention_df['total_customers']) * 100


# Write recipe outputs
df_tx_retention_seg = dataiku.Dataset("df_tx_retention_seg")
df_tx_retention_seg.write_with_schema(segment_retention_df)
